# Init + Auth

In [1]:
import requests
from pprint import pprint

In [2]:
AUTH_URL      = 'https://accounts.spotify.com/api/token'
BASE_URL      = 'https://api.spotify.com/v1/'

def get_spotify_access_token( authURL = AUTH_URL ):
    """ Get token from Spotify to allow API calls """
    # Original Author: https://stmorse.github.io/journal/spotify-api.html
    CLIENT_ID     = ""
    CLIENT_SECRET = ""

    with open( "../keys/spot_ID.txt" , 'r' ) as f:
        CLIENT_ID = f.readlines()[0].strip()
    
    with open( "../keys/spot_SECRET.txt" , 'r' ) as f:
        CLIENT_SECRET = f.readlines()[0].strip()
    
    if len( CLIENT_ID ) and len( CLIENT_SECRET ):
        print( "Authorization strings have been obtained!" )
    else:
        print( "Auth info is MISSING!" )

    # POST
    auth_response = requests.post( authURL, {
        'grant_type': 'client_credentials',
        'client_id': CLIENT_ID,
        'client_secret': CLIENT_SECRET,
    })
    auth_response_data = auth_response.json()
    access_token       = auth_response_data['access_token']
    return access_token

def get_headers( access_token ):
    """ Get header struct for Spotify API calls """
    return { 'Authorization': 'Bearer {token}'.format( token = access_token ) }

In [ ]:
access_token = get_spotify_access_token()

# Get Playlist Items

In [ ]:
playlist_id  = '6KI7A4MWrSM7EyKRUjxIi1'
response = requests.get( BASE_URL + 'playlists/' + playlist_id + '/tracks/', headers = get_headers( access_token ) )
response = response.json()

In [6]:
for key, val in response.items():
    print( key, ':', type( val ) )

href : <class 'str'>
items : <class 'list'>
limit : <class 'int'>
next : <class 'str'>
offset : <class 'int'>
previous : <class 'NoneType'>
total : <class 'int'>


In [10]:
print( f"Limit: {response['limit']}, Offset: {response['limit']}, Total: {response['limit']}, Length: {len(response['items'])}" )
print( response['next'] )

Limit: 100, Offset: 100, Total: 100, Length: 100
https://api.spotify.com/v1/playlists/6KI7A4MWrSM7EyKRUjxIi1/tracks?offset=100&limit=100


In [15]:
playlist_id  = '6KI7A4MWrSM7EyKRUjxIi1'
initURL      = BASE_URL + 'playlists/' + playlist_id + '/tracks/'
response     = requests.get( initURL, headers = get_headers( access_token ) ).json()
nextURL      = response['next']
playlistItms = response['items'][:]
while nextURL is not None:
    response = requests.get( nextURL, headers = get_headers( access_token ) ).json()
    nextURL  = response['next']
    playlistItms.extend( response['items'] )

print( f"Retrieved {len(playlistItms)} playlist items!" )
    




Retrieved 426 playlist items!


# Reorder Items

In [ ]:
# FIXME, START HERE: https://developer.spotify.com/documentation/web-api/reference/reorder-or-replace-playlists-tracks